In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("MinIO_Spark") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio-server:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .getOrCreate()

In [3]:
df = spark.read.parquet('fhvhv/2021/01/')

In [4]:
df.createOrReplaceTempView('data')

In [5]:
result = spark.sql('''
    SELECT 
        hvfhs_license_num, 
        COUNT(*) AS total_trips
    FROM data
    GROUP BY hvfhs_license_num
    ORDER BY total_trips DESC
''')

In [6]:
minio_path = "s3a://test/revenue.parquet"
result.write.parquet(minio_path, mode="overwrite")